# Treinando o algoritmo

Para o treinamento optamos por testar diferentes classes, cujos principais pontos positivos e negativos estão descritos abaixo:


### 1. **LightGBM**
  - **Por que usar:**  Muito rápido e eficiente em termos de uso de memória, especialmente em conjuntos de dados grandes, permite ajuste fino de muitos parâmetros, como a profundidade da árvore e o número de folhas, para melhorar o desempenho e lida bem com conjuntos de dados esparsos, o que é comum em problemas com muitas variáveis categóricas.
  - **Por que não usar LightGBM:** tem muitoshiperparâmetros que precisam ser ajustados corretamente, o que pode ser complexo e demorado e como outros algoritmos de gradient boosting, pode ser suscetível a overfitting, especialmente em conjuntos de dados pequenos ou mal balanceados.

### 2. **XGBoost (XGBClassifier)**
  - **Por que usar:** É  uma técnica de boosting extremamente eficiente e poderosa que geralmente supera muitos outros algoritmos em termos de performance em problemas de classificação. Ele é bom em capturar interações complexas e pode lidar com dados desbalanceados, que são comuns em problemas de churn.
  - **Por que não usar:** Similar ao Random Forest, XGBoost pode ser caro em termos computacionais. Além disso, requer um ajuste de hiperparâmetros mais cuidadoso para evitar overfitting, o que pode aumentar o tempo de desenvolvimento.

### 3. **Random Forest**
  - **Por que usar:** Robustex cobtra Overfiting, lida bem com dados faltantes
  - **Por que não usar:** Propensas a Bias

Aqui está um código que implementa os ajustes para `scale_pos_weight`, `sample_weight`, e o ajuste do threshold de decisão no XGBoost para lidar com dados desbalanceados:
### Explicação:
1. **Ajuste do `scale_pos_weight`** (ponto 2): Calculamos a razão entre as amostras negativas e positivas no conjunto de treino para balancear o impacto das classes.
2. **Uso de `sample_weight`** (ponto 4): Aplicamos pesos diferentes às amostras, dando mais importância às da classe minoritária.
3. **Threshold tuning** (ponto 5): Ao invés de usar o threshold padrão de 0.5 para a probabilidade de previsão, ajustamos para 0.3 para captar mais positivos.
Esse código pode ser adaptado ao seu problema específico, ajustando os parâmetros conforme necessário.

In [ ]:
# Calculando scale_pos_weight
num_pos = np.sum(y_train == 1)  # Número de positivos
num_neg = np.sum(y_train == 0)  # Número de negativos
scale_pos_weight = num_neg / num_pos

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
# Application of all Machine Learning methods
MLA = [
    LogisticRegression(class_weight='balanced',C=1),

    # LightGBM
    LGBMClassifier(scale_pos_weight=scale_pos_weight,  # Ajuste para classes desbalanceadas
   use_label_encoder=True),

    #Ensemble Methods
    XGBClassifier( scale_pos_weight=scale_pos_weight,  # Ajuste para classes desbalanceadas
   use_label_encoder=True),

    #Random Forest
    RandomForestClassifier(max_depth=2,class_weight = 'balanced')

    ]

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, roc_curve, auc, f1_score
from sklearn.model_selection import train_test_split, KFold
from sklearn import linear_model, ensemble, tree, naive_bayes
from sklearn.ensemble import RandomForestClassifier
import xgboost
import numpy as np


# Inicializando as colunas do DataFrame para comparação
MLA_columns = ['MLA used', 'Train Accuracy', 'Validation Accuracy', 'Test Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC']
MLA_compare = pd.DataFrame(columns=MLA_columns)

# Índice para linhas do DataFrame
row_index = 0

# Loop para cada algoritmo na lista MLA
for alg in MLA:
    print(f"Evaluando {alg.__class__.__name__}...")

    # Validação Cruzada usando o conjunto de treinamento
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    val_accuracies = []
    val_precisions = []
    val_recalls = []
    val_f1_scores = []
    val_auc_scores = []

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        # Treinando o modelo
        alg.fit(X_train_fold, y_train_fold)

        # Predições e métricas no conjunto de validação
        y_val_pred = alg.predict(X_val_fold)
        y_val_pred_proba = alg.predict_proba(X_val_fold)[:, 1] if hasattr(alg, 'predict_proba') else y_val_pred

        val_accuracies.append(alg.score(X_val_fold, y_val_fold))
        val_precisions.append(precision_score(y_val_fold, y_val_pred))
        val_recalls.append(recall_score(y_val_fold, y_val_pred))
        val_f1_scores.append(f1_score(y_val_fold, y_val_pred))
        fp, tp, _ = roc_curve(y_val_fold, y_val_pred_proba)
        val_auc_scores.append(auc(fp, tp))

    # Calculando métricas médias
    val_accuracy = round(np.mean(val_accuracies), 4)
    val_precision = round(np.mean(val_precisions), 4)
    val_recall = round(np.mean(val_recalls), 4)
    val_f1 = round(np.mean(val_f1_scores), 4)
    val_auc = round(np.mean(val_auc_scores), 4)

    # Treinamento final no conjunto de treino
    alg.fit(X_train, y_train)

    # Avaliação final no conjunto de teste
    y_test_pred = alg.predict(X_test)
    y_test_pred_proba = alg.predict_proba(X_test)[:, 1] if hasattr(alg, 'predict_proba') else y_test_pred

    test_accuracy = round(alg.score(X_test, y_test), 4)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    fp, tp, _ = roc_curve(y_test, y_test_pred_proba)
    auc_score = auc(fp, tp)

    # Armazenando métricas no DataFrame
    MLA_compare.loc[row_index] = [alg.__class__.__name__,
                                   round(alg.score(X_train, y_train), 4),
                                   val_accuracy,
                                   test_accuracy,
                                   precision,
                                   recall,
                                   f1,
                                   auc_score]

    row_index += 1

In [ ]:
MLA_compare.sort_values(by = ['Test Accuracy'], ascending = False, inplace = True)
MLA_compare